In [1]:
from llm.factory import LLMInterface
from setting.db import SessionLocal
from llm.embedding import get_text_embedding

# llm_client = LLMInterface("bedrock", "us.deepseek.r1-v1:0")
llm_client = LLMInterface("bedrock", "arn:aws:bedrock:us-east-1:841162690310:inference-profile/us.anthropic.claude-3-7-sonnet-20250219-v1:0")

In [2]:
from knowledgebase.doc_builder import DocBuilder

doc_builder = DocBuilder(llm_client, get_text_embedding)

sections = doc_builder.split_markdown_by_heading("../graph/docs/programming-guides/proto3.md", {
    "doc_version": "1.0",
    "category": "Programming Guides",
})

Source data created for ../graph/docs/programming-guides/proto3.md, id: 175f314b-0650-4ede-a874-da84621193de


In [ ]:
doc_builder.analyze_concepts()

In [5]:
from knowledgebase.doc_model import KnowledgeBlock
from setting.db import SessionLocal
from llm.embedding import get_text_embedding
from sqlalchemy.orm import aliased, defer, joinedload
from sqlalchemy import or_, asc

query_vector = get_text_embedding("How to update protobufmessage definition")

with SessionLocal() as db:
    blocks = db.query(
        KnowledgeBlock.name,
        KnowledgeBlock.content,
        KnowledgeBlock.content_vec.cosine_distance(
            query_vector
        ).label("embedding_distance"),
    ).order_by(asc("embedding_distance")).limit(3).all()

In [6]:
for block in blocks:
    print(block)

('Defining A Message Type {#simple}', '## Defining A Message Type {#simple}\n\nFirst let\'s look at a very simple example. Let\'s say you want to define a search\nrequest message format, w ... (13192 characters truncated) ... s for each language by following the\ntutorial for your chosen language. For even more API\ndetails, see the relevant [API reference](/reference/).\n', 0.5181685873025583)
('Updating A Message Type {#updating}', '## Updating A Message Type {#updating}\n\nIf an existing message type no longer meets all your needs – for example, you\'d\nlike the message format t ... (4727 characters truncated) ... emantically\n    identical result; however, clients using the `map` field definition may\n    reorder entries and drop entries with duplicate keys.\n', 0.5294359869773335)
('Using Other Message Types {#other}', '## Using Other Message Types {#other}\n\nYou can use other message types as field types. For example, let\'s say you\nwanted to include `Result` mess ... (2409 cha